In [4]:
import cv2
import numpy as np
from imutils.perspective import four_point_transform

In [5]:
ticket = cv2.imread('./ticket/ticket.jpg')
origin = ticket.copy()

ticket_gray = cv2.cvtColor(ticket, cv2.COLOR_BGR2GRAY)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2,2))
gray_eq = clahe.apply(ticket_gray)

_, thresh = cv2.threshold(gray_eq, 200, 255, cv2.THRESH_BINARY + cv2.THRESH_TRIANGLE)

cv2.imshow('thresh', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

kernel = np.ones((10,2), np.uint8)
open = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=6)
cv2.imshow('open', open)
cv2.waitKey(0)
cv2.destroyAllWindows()

kernel = np.ones((2,5), np.uint8)
open = cv2.morphologyEx(open, cv2.MORPH_OPEN, kernel, iterations=3)
cv2.imshow('open', open)
cv2.waitKey(0)
cv2.destroyAllWindows()

contours, _ = cv2.findContours(open, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
temp = ticket.copy()
cv2.drawContours(temp, contours, -1, (0, 255, 0), 1)
cv2.namedWindow('temp', 0)
cv2.resizeWindow('temp', 800, 800)
cv2.imshow('temp', temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

contours = sorted(contours, key=cv2.contourArea, reverse=True)[:1]
print(len(contours))
temp2 = ticket.copy()
cv2.drawContours(temp2, contours, -1, (0, 255, 0), 1)
cv2.namedWindow('temp2', 0)
cv2.resizeWindow('temp2', 800, 800)
cv2.imshow('temp2', temp2)
cv2.waitKey(0)
cv2.destroyAllWindows()

largest_contour = contours[0]

epsilon = 0.02 * cv2.arcLength(largest_contour, True)  # 误差阈值
approx = cv2.approxPolyDP(largest_contour, epsilon, True)  # 逼近曲线

temp3 = ticket.copy()

# 判断是否是四边形
if len(approx) == 4:
    print("找到四边形")
    cv2.drawContours(temp3, [approx], -1, (0, 255, 0), 2)  # 画出四边形
else:
    print(f"找到 {len(approx)} 个点，不是四边形")


cv2.namedWindow('Approximated Contour', 0)
cv2.resizeWindow('Approximated Contour', 800, 800)
# 显示结果
cv2.imshow("Approximated Contour", temp3)
cv2.waitKey(0)
cv2.destroyAllWindows()



1
找到四边形


In [6]:
warped = four_point_transform(origin, approx.reshape(4, 2))
cv2.namedWindow('warped', 0)
cv2.resizeWindow('warped', 600, 600)
cv2.imshow('warped', warped)
cv2.waitKey(0)
cv2.destroyAllWindows()